In [2]:
import numpy as np
from datetime import datetime
from scipy.sparse.linalg import eigsh
from susy_qm import calculate_wz_hamiltonian
from qiskit.quantum_info import SparsePauliOp

In [3]:
# Parameters
N = 3
a = 1.0
c = -0.8

potential = "linear"
#potential = 'quadratic'
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'
cutoff = 16
shots = 1024

In [56]:
# Calculate Hamiltonian and expected eigenvalues
ham_start_time = datetime.now()
H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c)
ham_calc_time = datetime.now() - ham_start_time
print(f"Time taken to create hamiltonian: {str(ham_calc_time)}")

hamiltonian = SparsePauliOp.from_operator(H)
hamiltonian

MemoryError: Unable to allocate 1.00 TiB for an array with shape (262144, 262144) and data type complex128

In [ ]:
starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

print(str(starttime))
print(f"Running for {N} sites")
print(f"Running for {boundary_condition} boundary conditions")
print(f"Running for {potential} potential, cutoff {cutoff}")

# Calculate Hamiltonian and expected eigenvalues
ham_start_time = datetime.now()
H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c)
ham_calc_time = datetime.now() - ham_start_time
print(f"Time taken to create hamiltonian: {str(ham_calc_time)}")

eig_start_time = datetime.now()
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = min(eigenvalues)
eig_calc_time = datetime.now() - eig_start_time
print(f"Time taken to find eigenvalues: {str(eig_calc_time)}")

2025-04-16_20-14-17
Running for 3 sites
Running for dirichlet boundary conditions
Running for linear potential, cutoff 16
Time taken to create hamiltonian: 0:00:00.196688


In [8]:
starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

print(str(starttime))
print(f"Running for {N} sites")
print(f"Running for {boundary_condition} boundary conditions")
print(f"Running for {potential} potential, cutoff {cutoff}")

# Calculate Hamiltonian and expected eigenvalues
ham_start_time = datetime.now()
H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c, to_dense=False)
ham_calc_time = datetime.now() - ham_start_time
print(f"Time taken to create hamiltonian: {str(ham_calc_time)}")

eig_start_time = datetime.now()
eigenvalue, eigenvector = eigsh(H, k=3, which='SA')
min_eigenvalue = eigenvalue[0]
min_eigenvector = eigenvector[:, 0]
eig_calc_time = datetime.now() - eig_start_time
print(f"Time taken to find eigenvalues: {str(eig_calc_time)}")

2025-04-16_11-07-51
Running for 3 sites
Running for dirichlet boundary conditions
Running for linear potential, cutoff 8
Time taken to create hamiltonian: 0:00:00.035093
Time taken to find eigenvalues: 0:00:02.125185


In [9]:
eigenvalue

array([2.73571762e-08, 1.00000003e+00, 9.99999556e-01])

In [18]:
from qutip import identity, sigmax, sigmay, sigmaz, Qobj, tensor
import numpy as np
from itertools import product

In [4]:
# Calculate Hamiltonian and expected eigenvalues
ham_start_time = datetime.now()
H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c, to_dense=False)
ham_calc_time = datetime.now() - ham_start_time
print(f"Time taken to create hamiltonian: {str(ham_calc_time)}")

Time taken to create hamiltonian: 0:00:00.166534


In [5]:
H

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 2982912 stored elements and shape (262144, 262144)>

In [42]:
import numpy as np
from scipy.sparse import csr_matrix, kron, identity
from itertools import product

# Define Pauli matrices as sparse
I = csr_matrix([[1, 0], [0, 1]], dtype=np.complex128)
X = csr_matrix([[0, 1], [1, 0]], dtype=np.complex128)
Y = csr_matrix([[0, -1j], [1j, 0]], dtype=np.complex128)
Z = csr_matrix([[1, 0], [0, -1]], dtype=np.complex128)

pauli_dict = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
labels = ['I', 'X', 'Y', 'Z']

def build_pauli_string(label):
    """Build sparse matrix for tensor product of Pauli label string."""
    result = pauli_dict[label[0]]
    for l in label[1:]:
        result = kron(result, pauli_dict[l], format='csr')
    return result

def decompose_sparse_hamiltonian(H_sparse, num_qubits):
    result = []
    for pauli_str in product(labels, repeat=num_qubits):
        label = ''.join(pauli_str)
        P = build_pauli_string(label)
        coeff = (P.conj().transpose().dot(H_sparse)).diagonal().sum() / (2 ** num_qubits)
        if abs(coeff) > 1e-10:
            result.append((label, coeff))
    return result

In [50]:
num_qubits = int(N * (np.log2(cutoff)+1))
decompose_sparse_hamiltonian(H,num_qubits)

[('IIIIII', np.complex128(3.375+0j)),
 ('IIIIXI', np.complex128(0.12074072828613334+0j)),
 ('IIIIXZ', np.complex128(-0.03235238063781505+0j)),
 ('IIIIZI', np.complex128(-0.5625+0j)),
 ('IIIIZZ', np.complex128(-0.5625+0j)),
 ('IIXIII', np.complex128(0.12074072828613336+0j)),
 ('IIXZII', np.complex128(-0.03235238063781504+0j)),
 ('IIZIII', np.complex128(-0.5625000000000001+0j)),
 ('IIZZII', np.complex128(-0.5625000000000002+0j)),
 ('IZIIII', np.complex128(0.5+0j)),
 ('XXIIII', np.complex128(0.25+0j)),
 ('YYIIII', np.complex128(0.25+0j)),
 ('ZIIIII', np.complex128(-0.5+0j))]

In [60]:
import numpy as np
from scipy.sparse import csr_matrix, kron, identity
from itertools import product
from joblib import Parallel, delayed

In [ ]:
# Define sparse Pauli matrices
I = csr_matrix([[1, 0], [0, 1]], dtype=np.complex128)
X = csr_matrix([[0, 1], [1, 0]], dtype=np.complex128)
Y = csr_matrix([[0, -1j], [1j, 0]], dtype=np.complex128)
Z = csr_matrix([[1, 0], [0, -1]], dtype=np.complex128)

pauli_dict = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
labels = ['I', 'X', 'Y', 'Z']

In [61]:
def build_pauli_string(label):
    """Build sparse matrix for tensor product of a Pauli label string."""
    result = pauli_dict[label[0]]
    for l in label[1:]:
        result = kron(result, pauli_dict[l], format='csr')
    return result

def _compute_pauli_coeff(label, H, norm_factor):
    P = build_pauli_string(label)
    coeff = (P.conj().transpose().dot(H)).diagonal().sum() / norm_factor
    if abs(coeff) > 1e-10:
        return (label, coeff)
    return None

def decompose_sparse_hamiltonian_parallel(H_sparse, num_qubits, n_jobs=6):
    norm_factor = 2 ** num_qubits
    all_labels = [''.join(p) for p in product(labels, repeat=num_qubits)]

    results = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(_compute_pauli_coeff)(label, H_sparse, norm_factor)
        for label in all_labels
    )

    # Filter out Nones
    return [r for r in results if r is not None]


In [ ]:
terms = decompose_sparse_hamiltonian_parallel(H, num_qubits)

for label, coeff in terms:
    print(f"{coeff:.3f} * {label}")